In [40]:
import SALib
from SALib.sample import saltelli
from SALib.analyze import sobol
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
#from tqdm import tqdm  # For progress bar
from ABM_Final_Final import SocialNetwork

In [41]:
# We define our variables and bounds
problem = {
    'num_vars': 8,
    'names': ['prob', 'w_pop', 'w_prox', 'w_sim', 'mu', 'temp', 'conf_mu', 'tol_mu'],
    'bounds': [[0., 1.], [0., 1.], [0., 1.], [0.01, 1.], [0.5,2], [0,1000], [0., 1.], [0.,1.]]
}

# Set the repetitions, the amount of steps, and the amount of distinct values per variable
replicates = 10
max_steps = 50
distinct_samples = 10

# We get all our samples here
param_values = saltelli.sample(problem, distinct_samples)

# Initialize DataFrame to store results
num_samples = len(param_values) * replicates

data = pd.DataFrame(index=range(num_samples), columns=['prob', 'w_pop', 'w_prox', 'w_sim', 'mu', 'temp', 'conf_mu', 'tol_mu'])
#print(data)
data['Run'], data['max IN degree'], data['avg clustering coeff'] = None, None, None

/var/folders/hk/3k38h_n53zl5ysg3__9h2b9r0000gn/T/ipykernel_41649/667753776.py:14: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, distinct_samples)
/Users/noaroebersen/.local/share/virtualenvs/ABM_Notebooks_0_1-qmu33wC9/lib/python3.9/site-packages/SALib/sample/saltelli.py:109: UserWarning: 
        Convergence properties of the Sobol' sequence is only valid if
        `N` (10) is equal to `2^n`.
        
  warnings.warn(msg)


In [49]:
# Define your model
def run_model(params,max_steps):
    n_agents = 100
    prob, w_pop, w_prox, w_sim, temp, mu, conf_mu, tol_mu = params
    model = SocialNetwork(n_agents, prob, w_pop, w_prox, w_sim, mu, temp, conf_mu, tol_mu)
    results = pd.DataFrame(model.Data_Collector)
    for k in range(max_steps + 1):
        model.step()
        print(f"\rProgress: {(k / max_steps) * 100:.2f}%", end='', flush=True)
   
    df_results = pd.DataFrame(model.Data_Collector)
    
    return df_results['avg clustering coeff'].iloc[-1], df_results['max IN degrees'].iloc[-1]

In [50]:
# Batch run
count = 1
for i in range(replicates):
    for vals in param_values:
        # Run the model with the current set of parameters
        print(f"\nRun {count}")
        clustering_coeff, max_in = run_model(vals,max_steps)
        data.iloc[count, 0:len(vals)] = list(vals)
        data.iloc[count, len(vals):len(vals)+3] = [count, clustering_coeff, max_in]
        count += 1
        
data.to_csv(f"sobol_rep{replicates}_maxstep{max_steps}_distsamples{distinct_samples}.csv", index=False)



Run 0
Progress: 100.00%
Run 1
Progress: 100.00%
Run 2
Progress: 100.00%
Run 3
Progress: 100.00%
Run 4
Progress: 100.00%
Run 5
Progress: 100.00%
Run 6
Progress: 100.00%
Run 7
Progress: 100.00%
Run 8
Progress: 100.00%
Run 9
Progress: 100.00%
Run 10
Progress: 100.00%
Run 11
Progress: 100.00%
Run 12
Progress: 100.00%
Run 13
Progress: 100.00%
Run 14
Progress: 100.00%
Run 15
Progress: 100.00%
Run 16
Progress: 100.00%
Run 17
Progress: 100.00%
Run 18
Progress: 100.00%
Run 19
Progress: 100.00%
Run 20
Progress: 100.00%
Run 21
Progress: 100.00%
Run 22
Progress: 100.00%
Run 23
Progress: 100.00%
Run 24
Progress: 100.00%
Run 25
Progress: 100.00%
Run 26
Progress: 100.00%
Run 27
Progress: 100.00%
Run 28
Progress: 100.00%
Run 29
Progress: 100.00%
Run 30
Progress: 100.00%
Run 31
Progress: 100.00%
Run 32
Progress: 100.00%
Run 33
Progress: 100.00%
Run 34
Progress: 100.00%
Run 35
Progress: 100.00%
Run 36
Progress: 100.00%
Run 37
Progress: 100.00%
Run 38
Progress: 100.00%
Run 39
Progress: 100.00%
Run 40
Pr

ValueError: The number of weights does not match the population

In [ ]:
Si_sheep = sobol.analyze(problem, data['avg clustering coeff'].values, print_to_console=True)
Si_wolves = sobol.analyze(problem, data['max IN degrees'].values, print_to_console=True)